In [7]:
from package import process_dataset
from package import encoder
from package import models

import numpy as np

from sklearn.model_selection import train_test_split
import pandas as pd

In [8]:
description = pd.read_csv("data/dicionario.csv", encoding='latin-1')
train = pd.read_csv("data/dados_treino_hackaton.csv", encoding='latin-1', index_col=False)
test = pd.read_csv("data/dados_teste_x_hackaton.csv", encoding='latin-1', index_col=False)

del train['Unnamed: 0']
del test['Unnamed: 0']

In [9]:
train = encoder.encode_DataFrame(train)
train = process_dataset.processColumns(train)

In [10]:
train_cols, target = process_dataset.generate_labels(train)

In [11]:
X = process_dataset.scaleData(train[train_cols], useSaved=False)
y = train[target]

In [12]:
#podemos variar o test_size de 0.1 até 0.2 ver como os modelos treinam melhor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)